In [1]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
ml_file = pd.read_pickle('table_totale_dum.pickle')
ml_file.head(3)

,tconst,primaryTitle,averageRating,vote_count,vote_average,revenue,budget,num_genres,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,inconnu,ratio_votes,director,director_fatc
0,tt0499549,Avatar,7.9,29045,7.600,2923706026,237000000,3,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3676.582278,James Cameron,0
1,tt4154796,Avengers: Endgame,8.4,23091,8.267,2794731755,356000000,3,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2748.928571,Action,1
2,tt1630029,Avatar: The Way of Water,7.6,7846,7.707,2319331580,460000000,3,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1032.368421,James Cameron,0


In [3]:
movie_file = pd.read_pickle('table_finale_dummies.pickle')
movie_file.head(3)

,tconst,primaryTitle,averageRating,vote_count,vote_average,revenue,budget,num_genres,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western,inconnu,ratio_votes,director_x,director_fact,originalTitle,startYear,genres_x,runtime,overview,cast,director_y,original_language,poster_path
0,tt0499549,Avatar,7.9,29045,7.600,2923706026,237000000,3,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3676.582278,James Cameron,0,Avatar,2009,"Action,Adventure,Fantasy",162,"In the 22nd century, a paraplegic Marine is di...",Sam Worthington,James Cameron,anglais,/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
1,tt4154796,Avengers: Endgame,8.4,23091,8.267,2794731755,356000000,3,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2748.928571,"Action, Adventure & Drama",1,Avengers: Endgame,2019,"Action,Adventure,Drama",181,After the devastating events of Avengers: Infi...,"Robert Downey Jr., Chris Evans",Inconnu,anglais,/or06FN3Dka5tukK1e9sl16pB3iy.jpg
2,tt1630029,Avatar: The Way of Water,7.6,7846,7.707,2319331580,460000000,3,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1032.368421,James Cameron,0,Avatar: The Way of Water,2022,"Action,Adventure,Fantasy",192,Set more than a decade after the events of the...,Sam Worthington,James Cameron,anglais,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg
